In [1]:
import sys

sys.path.append('../scripts')

In [2]:
import os

import pickle
import time
from multiprocessing import Pool
from time import sleep

import numpy as np
from tqdm.notebook import trange, tqdm
from scripts import centroids_graph_builder, graph_osm_loader, utils, clustering
from heapq import heappop, heappush
from itertools import count
import networkx as nx

import networkx as nx

In [43]:
def dijkstra_pfa_min_dst(graph: nx.Graph,
                 start: set[int],
                 ) -> \
        tuple[float, list[int]]:
    adjacency = graph._adj
    c = count()
    push = heappush
    pop = heappop
    dist = {}
    fringe = []
    for s in start:
        dist[s] = 0.0
        push(fringe, (0.0, next(c), s))
    while fringe:
        (d, _, v) = pop(fringe)
        for u, e in adjacency[v].items():
            vu_dist = d + e['length']
            if u not in dist or dist[u] > vu_dist:
                dist[u] = vu_dist
                push(fringe, (vu_dist, next(c), u))
    return dist


Примеры как резолвить комьюнити

In [67]:
GRAPH_ID = 'R2555133'  # R13470549 R2555133 R3766483

In [68]:
# примеры id есть в graph_osm_loader.py
g = graph_osm_loader.get_graph(GRAPH_ID)  # загрузка графа
# N = 1000
# p = 0.01
# g = get_rand_graph(N,p)
len(g.nodes), len(g.edges)

(17883, 26972)

In [69]:
utils.get_opt_cluster_count(len(g.nodes))  # оптимальное количество кластеров

858

In [70]:
g.remove_edges_from(nx.selfloop_edges(g))

In [71]:
name = 'cluster'

In [72]:
cms2 = clustering.resolve_k_means_communities(g, resolution=400, max_iteration=100, cluster_name=name, print_log=True)
print(len(cms2))

100%|██████████| 100/100 [00:05<00:00, 18.32it/s]

3107


In [73]:
cls2n = centroids_graph_builder.get_cls2n(g, name=name)  # мапа кластер к соседним кластерам 

In [74]:
g1, cls2c = centroids_graph_builder.build_center_graph(g, cms2, cls2n, log=True, name=name)

find edges: 100%|██████████| 3107/3107 [00:00<00:00, 5528.41it/s]


In [75]:
path = utils.get_path('pouits', f'points_{GRAPH_ID}.pickle')

if os.path.exists(path):
    with open(path, 'rb') as fp:
        points = pickle.load(fp)
        fp.close()
else:
    points = [utils.get_node_for_initial_graph_v2(g) for _ in trange(1000, desc='generate points')]
    with open(path, 'wb') as fp:
        pickle.dump(points, fp)
        fp.close()

In [ ]:
d_clusters = {}
d_nodes = {}
for u in tqdm(g1.nodes()):
    dst = dijkstra_pfa_min_dst(g, {cls2c[u]})
    for v in g1.nodes():
        d_clusters[u, v] = dst[cls2c[v]]
        d_clusters[v, u] = dst[cls2c[v]]

for u, d in tqdm(g.nodes(data=True)):
    c = cls2c[d[name]]
    l = nx.single_source_dijkstra(g, u, c, weight='length')[0]
    d_nodes[u] = l

  0%|          | 0/3107 [00:00<?, ?it/s]

In [ ]:
from scripts import ch

# chg = ch.build_ch_graph(g)

In [ ]:
nodes = g.nodes()

def func(u, v):
    c1 = nodes[u]['cluster']
    c2 = nodes[v]['cluster']   
    d1 = d_nodes[u] - d_nodes[v] - d_clusters[c1,c2]
    d2 = abs(d_clusters[c1,c2] - d_nodes[v]) - d_nodes[u] 
    d3 = abs(d_clusters[c1,c2] - d_nodes[u]) - d_nodes[v]
    d4 = d_nodes[v] - d_nodes[u] - d_clusters[c1,c2]
    return max(d1,d2,d3,d4)

In [ ]:
points = [utils.get_node_for_initial_graph_v2(g) for _ in trange(1000, desc='generate points')]

In [ ]:


def bi_dijkstra_pfa(graph: nx.Graph,
                    start: int,
                    end: int
                    ) -> tuple[float, list[int]]:
    if start == end:
        return 0, [start]
    push = heappush
    pop = heappop
    dist = ({start: (0, None)}, {end: (0, None)})
    fringe = ([], [])
    c = count()

    adjacency = graph._adj

    push(fringe[0], (0, next(c), 0, start))
    push(fringe[1], (0, next(c), 0, end))

    union_node = None
    union_dst = float('inf')
    while fringe[0] and fringe[1]:
        (_, _, d1, v1) = pop(fringe[0])
        (_, _, d2, v2) = pop(fringe[1])
        for u, e in adjacency[v1].items():
            vu_dist = d1 + e['length']
            if u not in dist[0] or dist[0][u][0] > vu_dist:
                dist[0][u] = (vu_dist, v1)
                push(fringe[0], (vu_dist + func(u, end), next(c), vu_dist, u))
                if u in dist[1]:
                    dd = dist[1][u][0] + dist[0][u][0]
                    if dd < union_dst:
                        union_dst = dd
                        union_node = u

        for u, e in adjacency[v2].items():
            vu_dist = d2 + e['length']
            if u not in dist[1] or dist[1][u][0] > vu_dist:
                dist[1][u] = (vu_dist, v2)
                push(fringe[1], (vu_dist + func(start, u), next(c), vu_dist, u))
                if u in dist[0]:
                    dd = dist[0][u][0] + dist[1][u][0]
                    if dd < union_dst:
                        union_dst = dd
                        union_node = u

        if union_node is not None:
            break

    d1, _ = dist[0][union_node]
    d2, _ = dist[1][union_node]
    path = []
    
    e = union_node
    while dist[0][e][1] is not None:
        path = [e] + path
        e = dist[0][e][1]
    path[0] = e

    e = union_node
    while dist[1][e][1] is not None:
        path+= [e]
        e = dist[1][e][1]
    path[-1] = e
    return union_dst, path


In [ ]:
def dijkstra_pfa_cls(graph: nx.Graph,
                     start: int,
                     end: int
                     ) -> tuple[float, list[int]]:
    if start == end:
        return 0, [start]
    push = heappush
    pop = heappop
    dist = {start: (0, None)}
    fringe = []
    c = count()
    adjacency = graph._adj
    push(fringe, (0, next(c), 0, func(start, end), start))

    while fringe:
        (_, _, d1, l1, v1) = pop(fringe) 
        if v1 == end:
            break
        for u, e in adjacency[v1].items():
            vu_dist = d1 + e['length']
            if u not in dist or dist[u][0] > vu_dist:
                dist[u] = (vu_dist, v1)
                lu = func(u, end)
                push(fringe, (vu_dist + lu, next(c), vu_dist, lu, u))

    path = [end]
    e = end
    while dist[e][1] is not None:
        e = dist[e][1]
        path = [e] + path
    l = 0
    e = g.edges()
    for i in range(len(path) - 1):
        p1, p2 = path[i], path[i + 1]
        l += e[p1, p2]['length']
    return l, path

In [66]:


NUM_ITERATION = 2  # чтобы уменьшить ошибку при вычислении времени выполнения, при каждом замере время меряется для NUM_ITERATION повторений
WORKER = 4  # количество потоков


def do_calc(data):
    pps, i = data

    stat = {}
    stat['l'] = []
    stat['h_l'] = []
    stat['p'] = []
    stat['h_p'] = []
    stat['time_l'] = []
    stat['time_h'] = []

    stat['delta'] = []
    # чисто чтобы tqdm нормально прогрузился 
    sleep(i / 10)
    print('start', i)

    for p1, p2 in tqdm(pps, desc='find paths', position=i):
        if (p1, p2) in stat:
            continue
        # класический дейкстра
        l, p = None, None
        start = time.time()
        for i in range(NUM_ITERATION):
            l, p = nx.single_source_dijkstra(g, p1, p2, weight='length')
        time_l = time.time() - start

        # иерархический
        h_l, h_p = None, None
        start = time.time()
        for _ in range(NUM_ITERATION):
            # h_l, h_p = bi_dijkstra_pfa(g, p1,p2) #find_path_length_h(g, g1,cms, p1, p2)
            h_l, h_p = dijkstra_pfa_cls(g, p1, p2)  #find_path_length_h(g, g1,cms, p1, p2)
            # h_l, h_p = nx.astar_path_length(g, p1, p2, heuristic=func, weight='length'),[]  #find_path_length_h(g, g1,cms, p1, p2)
            # h_l,h_p=ch_pfa(chg, p1,p2)
        time_h = time.time() - start

        delta = (h_l - l) / l * 100
        stat['l'].append(l)  # длина обычного пути
        stat['h_l'].append(h_l)  # длина иерархического пути
        stat['p'].append(p)  # обычный путь
        stat['h_p'].append(h_p)  # иерархический путь
        stat['delta'].append(delta)  # разница в длине
        stat['time_l'].append(time_l)  # обычное время 
        stat['time_h'].append(time_h)  # иерархическое
    return stat


data = [([p for p in points[i::WORKER]], i) for i in range(WORKER)]
# do_calc(data[0])
with Pool(WORKER) as p:
    res = p.map(do_calc, data)

stat = {}
for l in res:
    for d in l:
        if d not in stat:
            stat[d] = []
        stat[d].extend(l[d])
print(f"err_mean: {np.mean(stat['delta']):.2f}")
print(f"err_min: {np.min(stat['delta']):.2f}")
print(f"err_max: { np.max(stat['delta']):.2f}",)
print(np.mean(np.array(stat['time_l']) / np.array(stat['time_h'])))

start 0


find paths:   0%|          | 0/250 [00:00<?, ?it/s]

start 1


find paths:   0%|          | 0/250 [00:00<?, ?it/s]

start 2


find paths:   0%|          | 0/250 [00:00<?, ?it/s]

start 3


find paths:   0%|          | 0/250 [00:00<?, ?it/s]

err_mean: 0.00
err_min: 0.00
err_max: 0.00
2.719918710592404


In [ ]:
len(g.nodes), len(g.edges), nx.density(g)

In [ ]:
p1,p2 = points[0]